**The assembly API key is initialized.**

In [41]:
import json

api_key = ''
file_path = '/content/drive/MyDrive/GOOGLE COLAB/API.json'

try:
    with open(file_path, 'r') as file_to_read:
        json_data = json.load(file_to_read)
        api_key = json_data["API_KEY"]

except FileNotFoundError:
    print("JSON file not found at the specified path:", file_path)

except KeyError:
    print("API_KEY key not found in the JSON data.")

except json.JSONDecodeError as e:
    print("Error decoding JSON data:", str(e))

**Install python library for YouTube**

In [42]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Importing YouTube library from pytube. The user gets 2 options such that it can upload a youtube video or else it can upload video from the drive to get audio from the video.**

In [58]:
print('For Youtube video enter 1 and for drive video enter 0')
n=int(input('ENTER = '))
if n==1:
  link=input('ENTER THE LINK : ')
  from pytube import YouTube
  video=YouTube(link)
  yt=video.streams.get_audio_only()
  k=yt.download()
elif n==0:
  link=input('ENTER THE LINK : ')
  k=link
else:
  print('Sorry wrong input.')

For Youtube video enter 1 and for drive video enter 0
ENTER = 1
ENTER THE LINK : https://youtu.be/rgTGKcoq2Os


**Uploading the audio data retrieved from the YouTube video and feeding into the model to get the transcript.**

In [60]:
import sys
import time
import requests

file1 = k
def read_file(file1, size = 5242880):
  with open(file1, 'rb') as file:
    while True:
      data = file.read(size)
      if not data:
        break
      yield data
headers = {'authorization' : api_key}
response = requests.post('https://api.assemblyai.com/v2/upload', headers = headers, data = read_file(file1))
audio_url = response.json()['upload_url']
print(audio_url)

https://cdn.assemblyai.com/upload/e2e03b8f-3164-437d-ba2d-2306d7080cba


**This step performs authorization and perform transcript input response**

In [61]:
endpoint = "https://api.assemblyai.com/v2/transcript"

json = {"audio_url": audio_url}

headers = {"authorization" : api_key, "content_type": "application/json"}

transcript_input_response = requests.post(endpoint, json = json, headers = headers)

print(transcript_input_response.json())

{'id': '64s8yseu73-cf89-4b52-96ad-92555cd98b07', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'queued', 'audio_url': 'https://cdn.assemblyai.com/upload/e2e03b8f-3164-437d-ba2d-2306d7080cba', 'text': None, 'words': None, 'utterances': None, 'confidence': None, 'audio_duration': None, 'punctuate': True, 'format_text': True, 'dual_channel': None, 'webhook_url': None, 'webhook_status_code': None, 'webhook_auth': False, 'webhook_auth_header_name': None, 'speed_boost': False, 'auto_highlights_result': None, 'auto_highlights': False, 'audio_start_from': None, 'audio_end_at': None, 'word_boost': [], 'boost_param': None, 'filter_profanity': False, 'redact_pii': False, 'redact_pii_audio': False, 'redact_pii_audio_quality': None, 'redact_pii_policies': None, 'redact_pii_sub': None, 'speaker_labels': False, 'content_safety': False, 'iab_categories': False, 'content_safety_labels': {}, 'iab_categories_result': {}, 'language_dete

In [67]:
transcript_id = transcript_input_response.json()["id"]
transcript_id

'64s8yseu73-cf89-4b52-96ad-92555cd98b07'

**The transcript output status is reflected. The status reflected can be processing or completed.**

In [68]:
endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"
headers = {"authorization": api_key, }

transcript_output_response = requests.get(endpoint, headers = headers)
transcript_output_response.json()["status"]

'completed'

In [69]:
transcript_output_response.json()

{'id': '64s8yseu73-cf89-4b52-96ad-92555cd98b07',
 'language_model': 'assemblyai_default',
 'acoustic_model': 'assemblyai_default',
 'language_code': 'en_us',
 'status': 'completed',
 'audio_url': 'https://cdn.assemblyai.com/upload/e2e03b8f-3164-437d-ba2d-2306d7080cba',
 'text': "Here is the best voice assistant we've got. Apple, siri versus Google Assistant versus Amazon's. Alexa versus Samsung's bixby. And through 65 carefully selected questions, we're going to find out who can make your life the easiest. So let's start off in the study with some math questions. What's two plus two minus one? What's 104 plus 64 times 14? Too easy. Divide that by three. And nice. And now multiply that by seven. Convert six cups to milliliters. All of them. Got it. Split a bill of 69 pounds. 42 between three people. Oh, there's another dimension to this. Let's say that I'm finished with my studies and I want to turn off my smart light bulb. I can just say, turn off the light. And actually, every single 

**Gives the output of the transcript.**

In [70]:
transcript_output_response.json()["text"]

"Here is the best voice assistant we've got. Apple, siri versus Google Assistant versus Amazon's. Alexa versus Samsung's bixby. And through 65 carefully selected questions, we're going to find out who can make your life the easiest. So let's start off in the study with some math questions. What's two plus two minus one? What's 104 plus 64 times 14? Too easy. Divide that by three. And nice. And now multiply that by seven. Convert six cups to milliliters. All of them. Got it. Split a bill of 69 pounds. 42 between three people. Oh, there's another dimension to this. Let's say that I'm finished with my studies and I want to turn off my smart light bulb. I can just say, turn off the light. And actually, every single one of these assistants can do this, providing you have a light that's compatible with them. I bet you didn't expect Alexa to be winning this right now, but that was light work. So let's get a little more complicated. Let's say you're in your bedroom, you're winding down for the

**Enables user to download the transcript for the corresponding video.**

In [71]:
print('If you want to download enter 0 else enter 1 :- ')
m=int(input(''))
if m==0:
  transcript_file_path = 'Transcript'
  try:
    with open(transcript_file_path, 'w') as file1:
      file1.write(transcript_output_response.json()['text'])
    files.download(transcript_file_path)
  except OSError as e:
      print(f"An error occurred while opening the file: {e}")
elif m==1:
  print('You do not want to download.Thank You...')
else:
  print('Sorry Wrong input.')

If you want to download enter 0 else enter 1 :- 
0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>